In [1]:
import pandas as pd
import numpy as np
import os
import json,pickle
from collections import OrderedDict
from rdkit import Chem
from rdkit.Chem import MolFromSmiles
import networkx as nx
from utils import *


In [2]:
# f = open('./data/davis_train.csv')
# import pandas as pd
# a=pd.read_csv(f)

In [3]:
all_9_folds={}
for i in [0,1,2]:
    for j in [0,1,2]:
        file_name = 'fold' +str(i) +str(j) 
        
        temp = open('./data/davis/DAVIS_9_FOLDS/' + file_name +'.pkl', 'rb')
        new_df = pd.read_pickle(temp)
        all_9_folds.update({file_name:new_df})
        temp.close()
        

In [4]:
def create_test_train_davis(fold_number, all_9_folds):
    # fold_number is a string
    test_protein_fold_id = fold_number[0]
    test_ligand_fold_id = fold_number[1]
    test_set = pd.DataFrame(columns=['compound_iso_smiles', 'target_sequence', 'affinity'])
    train_set = pd.DataFrame(columns=test_set.columns)
    for i in [0,1,2]:
        for j in [0,1,2]:
            fold_name = 'fold' + str(i) + str(j)
            df = all_9_folds[fold_name]
            
#             if str(i) == test_protein_fold_id and str(j) == test_ligand_fold_id:
            if str(i) == test_protein_fold_id:
                new_df = pd.DataFrame(columns=test_set.columns)
                for index, row in df.iterrows():
                    smi = row['SMILES']
                    new_smi = Chem.MolToSmiles(Chem.MolFromSmiles(smi),isomericSmiles=True)
                    seq= row['Target Sequence']
                    label = row['Label']
                    new_df = new_df.append({'compound_iso_smiles':new_smi, 'target_sequence':seq,'affinity': label}, ignore_index=True)
                test_set = pd.concat([test_set,new_df], ignore_index=True)
#                 test_set = new_df.copy()
            else:
#             if str(i) != test_protein_fold_id and str(j) != test_ligand_fold_id:
                print(i,j)

                new_df = pd.DataFrame(columns=test_set.columns)
                for index, row in df.iterrows():
                    smi = row['SMILES']
                    new_smi = Chem.MolToSmiles(Chem.MolFromSmiles(smi),isomericSmiles=True)
                    seq= row['Target Sequence']
                    label = row['Label']
                    new_df = new_df.append({'compound_iso_smiles':new_smi, 'target_sequence':seq,'affinity': label}, ignore_index=True)
                train_set = pd.concat([train_set,new_df], ignore_index=True)
          
    return train_set, test_set

# Create train test split on these 9 folds
## fold_number is the id of fold. For example, test = fold00, train = fold 11,22,12,21

In [5]:
train, test = create_test_train_davis(fold_number='11', all_9_folds=all_9_folds)

0 0
0 1
0 2
2 0
2 1
2 2


In [6]:
test.to_csv('./data/davis_test.csv', index=False)
train.to_csv('./data/davis_train.csv', index=False)


In [16]:
train

,compound_iso_smiles,target_sequence,affinity
0,CCC1C(=O)N(C)c2cnc(Nc3ccc(C(=O)NC4CCN(C)CC4)cc...,MSDVAIVKEGWLHKRGEYIKTWRPRYFLLKNDGTFIGYKERPQDVD...,5.000000
1,Cn1c(Nc2ccc(C(F)(F)F)cc2)nc2cc(Oc3ccnc(-c4ncc(...,MSDVAIVKEGWLHKRGEYIKTWRPRYFLLKNDGTFIGYKERPQDVD...,5.000000
2,O=C(NOCC1CC1)c1ccc(F)c(F)c1Nc1ccc(I)cc1Cl,MSDVAIVKEGWLHKRGEYIKTWRPRYFLLKNDGTFIGYKERPQDVD...,5.000000
3,COc1cc2c(Oc3ccc4[nH]c(C)cc4c3F)ncnc2cc1OCCCN1C...,MSDVAIVKEGWLHKRGEYIKTWRPRYFLLKNDGTFIGYKERPQDVD...,5.000000
4,Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(N2CCN(C...,MSDVAIVKEGWLHKRGEYIKTWRPRYFLLKNDGTFIGYKERPQDVD...,5.000000
...,...,...,...
20055,Cn1cnc2c(F)c(Nc3ccc(Br)cc3Cl)c(C(=O)NOCCO)cc21,MDDKDIDKELRQKLNFSYCEETEIEGQKKVEESREASSQTPEKGEV...,5.000000
20056,COc1cc(N2CCC(N3CCN(C)CC3)CC2)ccc1Nc1ncc(Cl)c(N...,MDDKDIDKELRQKLNFSYCEETEIEGQKKVEESREASSQTPEKGEV...,5.000000
20057,Nc1nc(N)c2nc(-c3cccc(O)c3)c(-c3cccc(O)c3)nc2n1,MDDKDIDKELRQKLNFSYCEETEIEGQKKVEESREASSQTPEKGEV...,5.000000
20058,Cc1cnc(Nc2ccc(OCCN3CCCC3)cc2)nc1Nc1cccc(S(=O)(...,MDDKDIDKELRQKLNFSYCEETEIEGQKKVEESREASSQTPEKGEV...,5.221849


# To ensure that there are no common targets or drugs in train and test


In [8]:
# test_smiles = list(test['compound_iso_smiles'])
# test_targets = list(test['target_sequence'])
# train_smiles = list(train['compound_iso_smiles'])
# train_targets = list(train['target_sequence'])

# for i in test_smiles:
#     if i in train_smiles:
#         print("common entity present")
# for i in test_targets:
#     if i in train_targets:
#         print("common entity present")


In [9]:
# len(set(train_targets))

In [10]:
def atom_features(atom):
    return np.array(one_of_k_encoding_unk(atom.GetSymbol(),['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na','Ca', 'Fe', 'As', 'Al', 'I', 'B', 'V', 'K', 'Tl', 'Yb','Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn', 'H','Li', 'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr','Cr', 'Pt', 'Hg', 'Pb', 'Unknown']) +
                    one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5, 6,7,8,9,10]) +
                    one_of_k_encoding_unk(atom.GetTotalNumHs(), [0, 1, 2, 3, 4, 5, 6,7,8,9,10]) +
                    one_of_k_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5, 6,7,8,9,10]) +
                    [atom.GetIsAromatic()])

def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set{1}:".format(x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))

def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))

def smile_to_graph(smile):
    mol = Chem.MolFromSmiles(smile)
    
    c_size = mol.GetNumAtoms()
    
    features = []
    for atom in mol.GetAtoms():
        feature = atom_features(atom)
        features.append( feature / sum(feature) )

    edges = []
    for bond in mol.GetBonds():
        edges.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
    g = nx.Graph(edges).to_directed()
    edge_index = []
    for e1, e2 in g.edges:
        edge_index.append([e1, e2])
        
    return c_size, features, edge_index

def seq_cat(prot):
    x = np.zeros(max_seq_len)
    for i, ch in enumerate(prot[:max_seq_len]): 
        x[i] = seq_dict[ch]
    return x  



In [11]:
seq_voc = "ABCDEFGHIKLMNOPQRSTUVWXYZ"
seq_dict = {v:(i+1) for i,v in enumerate(seq_voc)}
seq_dict_len = len(seq_dict)
max_seq_len = 1000


In [12]:
compound_iso_smiles = []
for dt_name in ['davis']:
    opts = ['train','test']
    for opt in opts:
        df = pd.read_csv('data/' + dt_name + '_' + opt + '.csv')
        compound_iso_smiles += list( df['compound_iso_smiles'] )
compound_iso_smiles = set(compound_iso_smiles)
smile_graph = {}
for smile in compound_iso_smiles:
    g = smile_to_graph(smile)
    smile_graph[smile] = g

datasets = ['davis', 'kiba']
dataset = datasets[0]

In [13]:
# train set
df = pd.read_csv('data/' + dataset + '_train.csv')
train_drugs, train_prots,  train_Y = list(df['compound_iso_smiles']),list(df['target_sequence']),list(df['affinity'])
XT = [seq_cat(t) for t in train_prots]
train_drugs, train_prots,  train_Y = np.asarray(train_drugs), np.asarray(XT), np.asarray(train_Y)

# test set
df = pd.read_csv('data/' + dataset + '_test.csv')
test_drugs, test_prots,  test_Y = list(df['compound_iso_smiles']),list(df['target_sequence']),list(df['affinity'])
XT = [seq_cat(t) for t in test_prots]
test_drugs, test_prots,  test_Y = np.asarray(test_drugs), np.asarray(XT), np.asarray(test_Y)


In [14]:
processed_data_file_train = 'data/processed/' + dataset + '_train.pt'
processed_data_file_test = 'data/processed/' + dataset + '_test.pt'

In [15]:

# make data PyTorch Geometric ready
print('preparing ', dataset + '_train.pt in pytorch format!')
train_data = TestbedDataset(root='data', dataset=dataset+'_train', xd=train_drugs, xt=train_prots, y=train_Y,smile_graph=smile_graph)
print('preparing ', dataset + '_test.pt in pytorch format!')
test_data = TestbedDataset(root='data', dataset=dataset+'_test', xd=test_drugs, xt=test_prots, y=test_Y,smile_graph=smile_graph)
print(processed_data_file_train, ' and ', processed_data_file_test, ' have been created')

preparing  davis_train.pt in pytorch format!
Pre-processed data data/processed/davis_train.pt not found, doing pre-processing...
Converting SMILES to graph: 1/20060
Converting SMILES to graph: 2/20060
Converting SMILES to graph: 3/20060
Converting SMILES to graph: 4/20060
Converting SMILES to graph: 5/20060
Converting SMILES to graph: 6/20060
Converting SMILES to graph: 7/20060
Converting SMILES to graph: 8/20060
Converting SMILES to graph: 9/20060
Converting SMILES to graph: 10/20060
Converting SMILES to graph: 11/20060
Converting SMILES to graph: 12/20060
Converting SMILES to graph: 13/20060
Converting SMILES to graph: 14/20060
Converting SMILES to graph: 15/20060
Converting SMILES to graph: 16/20060
Converting SMILES to graph: 17/20060
Converting SMILES to graph: 18/20060
Converting SMILES to graph: 19/20060
Converting SMILES to graph: 20/20060
Converting SMILES to graph: 21/20060
Converting SMILES to graph: 22/20060
Converting SMILES to graph: 23/20060
Converting SMILES to graph: 2

Converting SMILES to graph: 244/20060
Converting SMILES to graph: 245/20060
Converting SMILES to graph: 246/20060
Converting SMILES to graph: 247/20060
Converting SMILES to graph: 248/20060
Converting SMILES to graph: 249/20060
Converting SMILES to graph: 250/20060
Converting SMILES to graph: 251/20060
Converting SMILES to graph: 252/20060
Converting SMILES to graph: 253/20060
Converting SMILES to graph: 254/20060
Converting SMILES to graph: 255/20060
Converting SMILES to graph: 256/20060
Converting SMILES to graph: 257/20060
Converting SMILES to graph: 258/20060
Converting SMILES to graph: 259/20060
Converting SMILES to graph: 260/20060
Converting SMILES to graph: 261/20060
Converting SMILES to graph: 262/20060
Converting SMILES to graph: 263/20060
Converting SMILES to graph: 264/20060
Converting SMILES to graph: 265/20060
Converting SMILES to graph: 266/20060
Converting SMILES to graph: 267/20060
Converting SMILES to graph: 268/20060
Converting SMILES to graph: 269/20060
Converting S

Converting SMILES to graph: 466/20060
Converting SMILES to graph: 467/20060
Converting SMILES to graph: 468/20060
Converting SMILES to graph: 469/20060
Converting SMILES to graph: 470/20060
Converting SMILES to graph: 471/20060
Converting SMILES to graph: 472/20060
Converting SMILES to graph: 473/20060
Converting SMILES to graph: 474/20060
Converting SMILES to graph: 475/20060
Converting SMILES to graph: 476/20060
Converting SMILES to graph: 477/20060
Converting SMILES to graph: 478/20060
Converting SMILES to graph: 479/20060
Converting SMILES to graph: 480/20060
Converting SMILES to graph: 481/20060
Converting SMILES to graph: 482/20060
Converting SMILES to graph: 483/20060
Converting SMILES to graph: 484/20060
Converting SMILES to graph: 485/20060
Converting SMILES to graph: 486/20060
Converting SMILES to graph: 487/20060
Converting SMILES to graph: 488/20060
Converting SMILES to graph: 489/20060
Converting SMILES to graph: 490/20060
Converting SMILES to graph: 491/20060
Converting S

Converting SMILES to graph: 751/20060
Converting SMILES to graph: 752/20060
Converting SMILES to graph: 753/20060
Converting SMILES to graph: 754/20060
Converting SMILES to graph: 755/20060
Converting SMILES to graph: 756/20060
Converting SMILES to graph: 757/20060
Converting SMILES to graph: 758/20060
Converting SMILES to graph: 759/20060
Converting SMILES to graph: 760/20060
Converting SMILES to graph: 761/20060
Converting SMILES to graph: 762/20060
Converting SMILES to graph: 763/20060
Converting SMILES to graph: 764/20060
Converting SMILES to graph: 765/20060
Converting SMILES to graph: 766/20060
Converting SMILES to graph: 767/20060
Converting SMILES to graph: 768/20060
Converting SMILES to graph: 769/20060
Converting SMILES to graph: 770/20060
Converting SMILES to graph: 771/20060
Converting SMILES to graph: 772/20060
Converting SMILES to graph: 773/20060
Converting SMILES to graph: 774/20060
Converting SMILES to graph: 775/20060
Converting SMILES to graph: 776/20060
Converting S

Converting SMILES to graph: 1001/20060
Converting SMILES to graph: 1002/20060
Converting SMILES to graph: 1003/20060
Converting SMILES to graph: 1004/20060
Converting SMILES to graph: 1005/20060
Converting SMILES to graph: 1006/20060
Converting SMILES to graph: 1007/20060
Converting SMILES to graph: 1008/20060
Converting SMILES to graph: 1009/20060
Converting SMILES to graph: 1010/20060
Converting SMILES to graph: 1011/20060
Converting SMILES to graph: 1012/20060
Converting SMILES to graph: 1013/20060
Converting SMILES to graph: 1014/20060
Converting SMILES to graph: 1015/20060
Converting SMILES to graph: 1016/20060
Converting SMILES to graph: 1017/20060
Converting SMILES to graph: 1018/20060
Converting SMILES to graph: 1019/20060
Converting SMILES to graph: 1020/20060
Converting SMILES to graph: 1021/20060
Converting SMILES to graph: 1022/20060
Converting SMILES to graph: 1023/20060
Converting SMILES to graph: 1024/20060
Converting SMILES to graph: 1025/20060
Converting SMILES to grap

...

Converting SMILES to graph: 8959/9996
Converting SMILES to graph: 8960/9996
Converting SMILES to graph: 8961/9996
Converting SMILES to graph: 8962/9996
Converting SMILES to graph: 8963/9996
Converting SMILES to graph: 8964/9996
Converting SMILES to graph: 8965/9996
Converting SMILES to graph: 8966/9996
Converting SMILES to graph: 8967/9996
Converting SMILES to graph: 8968/9996
Converting SMILES to graph: 8969/9996
Converting SMILES to graph: 8970/9996
Converting SMILES to graph: 8971/9996
Converting SMILES to graph: 8972/9996
Converting SMILES to graph: 8973/9996
Converting SMILES to graph: 8974/9996
Converting SMILES to graph: 8975/9996
Converting SMILES to graph: 8976/9996
Converting SMILES to graph: 8977/9996
Converting SMILES to graph: 8978/9996
Converting SMILES to graph: 8979/9996
Converting SMILES to graph: 8980/9996
Converting SMILES to graph: 8981/9996
Converting SMILES to graph: 8982/9996
Converting SMILES to graph: 8983/9996
Converting SMILES to graph: 8984/9996
Converting S

Converting SMILES to graph: 9201/9996
Converting SMILES to graph: 9202/9996
Converting SMILES to graph: 9203/9996
Converting SMILES to graph: 9204/9996
Converting SMILES to graph: 9205/9996
Converting SMILES to graph: 9206/9996
Converting SMILES to graph: 9207/9996
Converting SMILES to graph: 9208/9996
Converting SMILES to graph: 9209/9996
Converting SMILES to graph: 9210/9996
Converting SMILES to graph: 9211/9996
Converting SMILES to graph: 9212/9996
Converting SMILES to graph: 9213/9996
Converting SMILES to graph: 9214/9996
Converting SMILES to graph: 9215/9996
Converting SMILES to graph: 9216/9996
Converting SMILES to graph: 9217/9996
Converting SMILES to graph: 9218/9996
Converting SMILES to graph: 9219/9996
Converting SMILES to graph: 9220/9996
Converting SMILES to graph: 9221/9996
Converting SMILES to graph: 9222/9996
Converting SMILES to graph: 9223/9996
Converting SMILES to graph: 9224/9996
Converting SMILES to graph: 9225/9996
Converting SMILES to graph: 9226/9996
Converting S

Converting SMILES to graph: 9446/9996
Converting SMILES to graph: 9447/9996
Converting SMILES to graph: 9448/9996
Converting SMILES to graph: 9449/9996
Converting SMILES to graph: 9450/9996
Converting SMILES to graph: 9451/9996
Converting SMILES to graph: 9452/9996
Converting SMILES to graph: 9453/9996
Converting SMILES to graph: 9454/9996
Converting SMILES to graph: 9455/9996
Converting SMILES to graph: 9456/9996
Converting SMILES to graph: 9457/9996
Converting SMILES to graph: 9458/9996
Converting SMILES to graph: 9459/9996
Converting SMILES to graph: 9460/9996
Converting SMILES to graph: 9461/9996
Converting SMILES to graph: 9462/9996
Converting SMILES to graph: 9463/9996
Converting SMILES to graph: 9464/9996
Converting SMILES to graph: 9465/9996
Converting SMILES to graph: 9466/9996
Converting SMILES to graph: 9467/9996
Converting SMILES to graph: 9468/9996
Converting SMILES to graph: 9469/9996
Converting SMILES to graph: 9470/9996
Converting SMILES to graph: 9471/9996
Converting S

Converting SMILES to graph: 9736/9996
Converting SMILES to graph: 9737/9996
Converting SMILES to graph: 9738/9996
Converting SMILES to graph: 9739/9996
Converting SMILES to graph: 9740/9996
Converting SMILES to graph: 9741/9996
Converting SMILES to graph: 9742/9996
Converting SMILES to graph: 9743/9996
Converting SMILES to graph: 9744/9996
Converting SMILES to graph: 9745/9996
Converting SMILES to graph: 9746/9996
Converting SMILES to graph: 9747/9996
Converting SMILES to graph: 9748/9996
Converting SMILES to graph: 9749/9996
Converting SMILES to graph: 9750/9996
Converting SMILES to graph: 9751/9996
Converting SMILES to graph: 9752/9996
Converting SMILES to graph: 9753/9996
Converting SMILES to graph: 9754/9996
Converting SMILES to graph: 9755/9996
Converting SMILES to graph: 9756/9996
Converting SMILES to graph: 9757/9996
Converting SMILES to graph: 9758/9996
Converting SMILES to graph: 9759/9996
Converting SMILES to graph: 9760/9996
Converting SMILES to graph: 9761/9996
Converting S

Converting SMILES to graph: 9981/9996
Converting SMILES to graph: 9982/9996
Converting SMILES to graph: 9983/9996
Converting SMILES to graph: 9984/9996
Converting SMILES to graph: 9985/9996
Converting SMILES to graph: 9986/9996
Converting SMILES to graph: 9987/9996
Converting SMILES to graph: 9988/9996
Converting SMILES to graph: 9989/9996
Converting SMILES to graph: 9990/9996
Converting SMILES to graph: 9991/9996
Converting SMILES to graph: 9992/9996
Converting SMILES to graph: 9993/9996
Converting SMILES to graph: 9994/9996
Converting SMILES to graph: 9995/9996
Converting SMILES to graph: 9996/9996
Graph construction done. Saving to file.
data/processed/davis_train.pt  and  data/processed/davis_test.pt  have been created


# With this the processing of data is done for the specific proposed davis fold
## so now we can proceed to train and test on these models by running training.py